# 귀여운 고양이 노노그램 그림을 찾아주는 유전알고리즘 코드

(10*10의 격자판을 기준으로 진행했습니다.)

In [264]:
# 확률적 요소(돌연변이, 1세대 무작위 생성 등)가 사용되기에 random 모듈을 사용합니다.
import random

In [265]:
# 0과1을 섞어서 개체(문자열) 1개를 생성해주는 함수 (이걸 반복적으로 돌려서 최초 세대(개체들)을 생성할수 있습니다.)
def 무작위_문자열생성기():
    
    # 0과 1을 중복을 허용(random.choice의 기능)하면서 10줄의 문자열을 생성하는 과정
    Word = "".join(random.choice(['0','1']) for _ in range(10))
    
    # 무작위로 생성된 문자열을 반환
    return Word

#print(무작위_문자열생성기())   #test case 입니다.(여러번 돌려본 결과 값이 다 다르게 나오는걸 알수 있음!)

In [266]:
# 최초의 개체들을 무작위로 생성해주는 함수(조상님들)
def 조상님_생성기(cnt):    # cnt: 한 세대에 생성되는 개체수를 의미
    # 개체들을 담을 리스트 초기화
    First_Generation = [] 
    
    for _ in range(cnt): # size 만큼 loop를 돌면서
        First_Generation.append(무작위_문자열생성기())
    
    # 조상님 리스트를 반환 한다.
    return First_Generation

#print(조상님_생성기(10)) #test case 입니다.(여러번 돌려본 결과 무작위의 10개의 개체들이 담겨있는걸 알수 있다.)

In [267]:
# 적응도 점수 계산 하는 함수 (이 점수를 바탕으로 교배에 우선순위가 갈린다!)
def 적응도계산(최적해, 후보문자열):
    # 점수 담을 변수를 초기화
    score = 0
    
    # 한글자 한글자씩 비교해서 일치하면? 1점을 획득하는 방식 (10점이면 정답!)
    for i in range(10):
        #print(i, 최적해, 후보문자열, '오류발생포인트')
        if 최적해[i] == 후보문자열[i]:
            score += 1
    
    # 0~10사이의 적응도 점수를 반환합니다.
    return score

#print(적응도계산('1111111111', '1010101010'))     #testcase 입니다.(앞의 11111만 일치 하므로 5점 계산완료!)

In [268]:
# 한 세대의 개체들중에서 최적해에 가장 근접한 순으로 리스트를 반환합니다.
def 개체들의적응도계산결과(개체들, 찾고자하는최적해):
    #[후보문자열, 점수] 꼴로 원소들을 담아줄 리스트를 초기화
    Toprank_score_list = []
    
    for 후보문자열 in 개체들:
        # 각각의 자식들의 점수를 측정한다.
        score = 적응도계산(찾고자하는최적해, 후보문자열)   
        
        Toprank_score_list.append([후보문자열, score])     #[예상문자열 ,점수]

    # key(score)를 기준으로 내림차순으로 정렬한다. 
    # 0번인덱스에 위치한 [예상문자열 ,점수]가 가장 강한개체, -1번 인덱스에 위치한 개체가 가장 약한개체를 의미한다.
    Toprank_score = sorted(Toprank_score_list, key = lambda q: q[1], reverse=True)
    return Toprank_score

# testcase 입니다. (무작위 10개의 문자열이 담긴 배열을 만들고, 각 문자열들이 '1111100000'과 비교해서 나온 점수를 바탕으로 내림차순 정렬 완료)
# N = 조상님_생성기(10)
# print(개체들의적응도계산결과(N, '1111100000'))

In [269]:
# 적응도 평가도 끝난뒤, 어떤 개체가 살아남아서 교배를 할수 있을지를 결정하는 함수
def 최종생존개체(개체들의적응도계산결과_리스트, 우수개체수, 운좋게살아남는개체수):
    Next_Generation = []

    # best_sample 수 만큼 뽑아서 다음 세대로 보낸다.
    for i in range(우수개체수):
        Next_Generation.append(개체들의적응도계산결과_리스트[i][0])
    
    # 랜덤으로 운이 좋은 자식들을 살린다.(sample은 중복을 허용하지 않고 선택하는 경우의수)
    운좋은개체들 = random.sample(개체들의적응도계산결과_리스트, 운좋게살아남는개체수)
    for R in 운좋은개체들:
        Next_Generation.append(R[0])

    # 필요 없는 부분
    # # 다음 세대의 아이들의 수가 부족할 수 있기 때문에 랜덤으로 생성하여 채워넣는 부분이다.
    # if len(next_generation) < best_sample + lucky_few:
    #     Next_Generation.append(무작위_문자열생성기(10))    # 무작위로 생성되는 부분(짝수 개수로 맞추기 위해서)

    # 다음세대 개체들 리스트 순서를 무작위로 뒤섞고 반환하기
    random.shuffle(Next_Generation)
    return Next_Generation

# testcase 입니다. (적응도 점수 계산을 마친10개의 문자열중 누가 살아남는지가 결정 됩니다.)
# N = 조상님_생성기(10)
# N_list = 개체들의적응도계산결과(N, '1111100000')
# print(최종생존개체(N_list, 7,3))    # 적자생존 결과 살아남는 개체들 (우수개체7마리, 운좋은개체3마리로 설정함)


In [270]:
# 이제 적자생존이 끝난뒤, 교배후 자식을 생성하는 함수
def 자식생성기(아버지, 어머니):    # 살아남은 문자열 넣어 주면 된다.
    자식문자열 = ''
    
    # 우리가 배운 병렬적으로 진행 하는 방식 보다, 
    # 실제 인간의 유전자 처럼 확률적으로 어머니, 아버지 유전자를 가져간다(낮은 확률로 어머니랑 똑같거나, 아버지랑 똑같은 경우가 생길수도 있는 재미있는 아이디어)
    
    for i in range(10):
        # 50%의 확률로 아버지 유전자 한개 획득
        if (random.randint(1,101) < 50): 
            자식문자열 += 아버지[i]
        # 나머지 50% 확률로 어머니의 유전자를 획득
        else:
            자식문자열 += 어머니[i]
    return 자식문자열

# testcase 입니다. (아버지, 어머니의 유전자가 섞여서 자식들이 나오는걸 알수 있습니다. 알페벳, 숫자가 무작위로 섞여서 나옴)
# print(자식생성기('abcdefghij', '0123456789'))

In [271]:
# ex) 6명의 부모가 생존, 이때 3명의 커플이 1명의 자식만 생성하면하는 경우...
# 개체군의 감소폭이 너무 빠르다고 생각, 우리팀은 그래서 부모들이 2명씩 자식을 만든다고 가정헀습니다.
def 자식들을생성(살아남은부모들, 한커플당만드는자식수):
    children= []
    
    # 아래 코드를 알기 쉽게 예시를 들어보겠습니다.
    # [a,b,c,d,e,f,g,h,i] 의 개체들이 있다고 하면, a와i, b와h, c와g가 만나서 자식을 생성하는 원리입니다.(이떄 서로 다른 2개의 자식이 생성 됩니다.)
    for i in range(int(len(살아남은부모들)/2)):
        for _ in range(한커플당만드는자식수):    
            children.append(자식생성기(살아남은부모들[i], 살아남은부모들[len(살아남은부모들)-1-i]))
            
    return children

# testcase 입니다. (교배후 자식개체들이 담긴 문자열을 반환 합니다.)
# N = 조상님_생성기(10)
# N_list = 개체들의적응도계산결과(N, '1111100000')    # 1111100000은 임시적으로 사용하는 최적해 이다.
# N_survive=최종생존개체(N_list, 7,3)    # 적자생존 결과 살아남는 개체들 (우수개체 7마리, 운좋은개체 3마리로 설정함)
# print(자식들을생성(N_survive, 2))     # 한 커플당 2개의 자식을 만든다.

In [272]:
# 돌연변이(문자열을 구성하는 문자열중 1개가 바뀜)를 만들어 주는 함수
def 돌연변이(개체):    # 돌연변이를 원하는 문자열 한개를 넣어주면 된다.
    # 0~9사이의 무작위 인덱스 번호를 선택한다.
    index = int (random.random() * len(개체)) 
    
    # 첫번째 칸이 돌연변이로 선택된 경우
    if index == 0:   # 첫번째 글자면 처리하는 경우
        # 0이면 1로 바꿔줌
        if 개체[index] =='0':
            개체 = '1' + 개체[1:]
        # 1이면 0으로 바꿔줌
        else:
            개체 = '0' + 개체[1:]
    
    # 양끝의 사이에 위치한 칸이 선택된경우
    else :
        if 개체[index]=='0':
            개체 = 개체[:index] + '1' + 개체[index+1:]
        else:
            개체 = 개체[:index] + '0' + 개체[index+1:]


    return 개체

# testcase 입니다(무작위 위치의 한개의 글자가 바뀌는걸 볼수 있습니다.)
print(돌연변이('0000000000'))

0000000010


In [273]:
# 살아 남은 아이들중에서 10퍼센트 확률로 돌연변이를 생성한다.
def 돌연변이개체생성(자식개체들, 변이확률):     # 문자열 2차원 리스트, 돌연변이 확률(정수) 10으로 입력하면 10퍼센트 확률임
    for i in range(len(자식개체들)):
        if random.random() * 100 < 변이확률:
            자식개체들[i] = 돌연변이(자식개체들[i])
    return 자식개체들

# test 케이스 입니다. ( 한 세대마다 아래의 작업이 반복됩니다)
N = 조상님_생성기(10)
N_list = 개체들의적응도계산결과(N, '1111100000')
N_survive=최종생존개체(N_list, 7,3)    # 적자생존 결과 살아남는 개체들 (우수개체7마리, 운좋은개체3마리로 설정함)
N_children = 자식들을생성(N_survive, 2)    # 한 커플단 2개의 자식을 만든다.
print(돌연변이개체생성(N_children, 10))    #10퍼센트 확률로 자식개체의 돌연변이를 생성 한다.

['1100010100', '1101010101', '0111010000', '0010001100', '0111110011', '0111110011', '1101111010', '0001111010', '0111100110', '1111100100']


## <이제 함수 구현을 마치고 main 함수를 작성하는 부분 입니다!>

In [274]:
# 고양이 노노그램 그림을 문자열 형태로 표현한다.(각 열마다 0이면 빈칸, 1이면 색칠된 칸 형태 입니다.)

Love_cat = ['1111001110','0101011111','0111111111','0101011111','1111001111','0000000111','0000000011','0000000110','0010001100','0001111000']

# 한개의 열마다 찾아낸 근사해를 저장하는 리스트 (시각화 할때 사용할 예정 입니다.)
Local_goal =[]


# colum은 찾고자 하는 최적해를 의미 한다.
for colum in Love_cat:
    
    세대발생수 = 10
    초기개체수 = 10
    우수한개체선택 = 7
    운좋게선택되는개체 = 3
    돌연변이확률 = 10
    조상=조상님_생성기(초기개체수)
    for _ in range(세대발생수):
        적응도점수계산완료 = 개체들의적응도계산결과(조상, colum)
        
        #이미 최적해를 찾으면 True로 바뀐다.
        check_num=True
        
        # 가장 높은 점수가 10이면 정답을 찾은 것이다.
        if 적응도점수계산완료[0][1] ==10:
            print(f'{_+1} 세대에서 최적해 발견!: ',적응도점수계산완료[0][0])
            Local_goal.append(적응도점수계산완료[0][0])
            
            # 최적해를 찾았으므로, False로 바꿔 줌
            check_num=False
            break
        
        # 여기서부터는 최적해를 못찾을때 근사해라도 찾는 과정(진화의 시작)
        생존자들 = 최종생존개체(적응도점수계산완료, 우수한개체선택, 운좋게선택되는개체)
        
        #print('생존자', survivors)
        # 자식 생성 (2쌍이 2마리를 만든다.)
        자식 = 자식들을생성(생존자들, 2)
        
        # 돌연변이 까지 마친후 다음 세대가 생성된다.
        신세대개체들 =돌연변이개체생성(자식, 돌연변이확률)
        
        조상 = 신세대개체들   # 구세대를 신세대가 교체 한다.
        
        print(f"### {(_+1)}세대 최강개체 정보: {적응도점수계산완료[0][0]}, 적응도점수: {적응도점수계산완료[0][1]} ###")

    # 만약 이번 세대에서 최적해가 들어갔다면 추가하지 않는다.
    if check_num:
        Local_goal.append(적응도점수계산완료[0][0])    # 모든 세대중 가장 최적해와 유사한 근사해를 정답 리스트에 넣는다.
    print("한 세대의 끝(한개의 열의 LocalSearch를 마침!)*****************************************************")    

### 1세대 최강개체 정보: 1101010010, 적응도점수: 6 ###
### 2세대 최강개체 정보: 1010101110, 적응도점수: 7 ###
### 3세대 최강개체 정보: 1101011110, 적응도점수: 8 ###
### 4세대 최강개체 정보: 1110001111, 적응도점수: 8 ###
### 5세대 최강개체 정보: 1110011110, 적응도점수: 8 ###
### 6세대 최강개체 정보: 1110101110, 적응도점수: 8 ###
### 7세대 최강개체 정보: 1101001110, 적응도점수: 9 ###
### 8세대 최강개체 정보: 1110001110, 적응도점수: 9 ###
### 9세대 최강개체 정보: 1110001110, 적응도점수: 9 ###
### 10세대 최강개체 정보: 1110001110, 적응도점수: 9 ###
한 세대의 끝(한개의 열의 LocalSearch를 마침!)*****************************************************
### 1세대 최강개체 정보: 1011011111, 적응도점수: 7 ###
### 2세대 최강개체 정보: 1011011110, 적응도점수: 6 ###
### 3세대 최강개체 정보: 0111011010, 적응도점수: 7 ###
### 4세대 최강개체 정보: 0011011010, 적응도점수: 6 ###
### 5세대 최강개체 정보: 0001011010, 적응도점수: 7 ###
### 6세대 최강개체 정보: 0011011010, 적응도점수: 6 ###
### 7세대 최강개체 정보: 0011011011, 적응도점수: 7 ###
### 8세대 최강개체 정보: 0011011011, 적응도점수: 7 ###
### 9세대 최강개체 정보: 0011011011, 적응도점수: 7 ###
### 10세대 최강개체 정보: 0011011011, 적응도점수: 7 ###
한 세대의 끝(한개의 열의 LocalSearch를 마침!)****************************************

In [275]:
# 이제 국지적 탐색을 마친뒤 우리가 원하는 고양이 노노그램 그림이 나오는지 확인 
# (이미지가 생각보다 작게 나옵니다. 보고서에는 확대본을 첨부하였습니다!)
print('유전 알고리즘을 통해 얻은 각 열의 색칠정보(근사해)리스트: ', Local_goal)

# 10*10 격자판으로 시각화를 시작! (완벽하지는 않지만, 그래도 그럴싸한 고양이 노노그램 그림을 반환해주는걸 알수 있습니다!)
from PIL import Image
def create_nonogram_image(data, size):
    # data: 0과 1로 이루어진 리스트
    # size: 이미지 크기 (가로, 세로)
    image = Image.new('RGB', size)
    pixels = image.load()
    for x in range(size[0]):
        for y in range(size[1]):
            # 1이면 검정색, 0이면 흰색을 출력 한다.
            if data[x][y] == 1:
                pixels[x, y] = (0, 0, 0) # 검정색 RGB
            else:
                pixels[x, y] = (255, 255, 255) # 흰색 RGB
    return image

def visualize_nonogram(data):
    # data: 0과 1로 이루어진 리스트
    size = (10,10)
    image = create_nonogram_image(data, size)
    image.show()

# 해집합 리스트는 문자열로 되어 있으므로, 각각 정수형으로 변환해주면 시각화가 가능하다.
data = [[int(x) for x in s] for s in Local_goal]
# 시각화
visualize_nonogram(data)

유전 알고리즘을 통해 얻은 각 열의 색칠정보(근사해)리스트:  ['1110001110', '0011011011', '1111011111', '0011001111', '1111001111', '0000000111', '0000110011', '0000000111', '0010001000', '0001111000']
